# Locate & return indices

In [1]:
import os
workdir = '/data/hyeryung/mucoco'
print("Current dir: ", os.getcwd())
print("Changing directory...")
os.chdir(workdir)
print("Current dir: ", os.getcwd())
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import torch
from torch.nn import Softmax
from torch.utils.data import DataLoader

from notebooks.utils.load_ckpt import define_model

import json
import argparse
import time
from itertools import chain
from tqdm import tqdm


Current dir:  /data/hyeryung/mucoco/notebooks/locate
Changing directory...
Current dir:  /data/hyeryung/mucoco


In [2]:
BATCH_SIZE=64
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device: ", DEVICE)

device:  cuda


In [3]:
## 
print("num available devices: ", torch.cuda.device_count())

num available devices:  4


In [4]:
DEVICE = torch.device('cpu')

### Prototype

In [3]:
# 원본 데이터 로드
samples=pd.read_csv('./notebooks/results/test_mucoco+add_preds.csv')
# label이 1인 데이터만 사용
sample1=samples.loc[samples['toxicity']>0.5].copy()

In [7]:
# load trained model
ckpt_path='/home/hyeryungson/mucoco/models/models_balanced/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
# ckpt_path='/home/hyeryungson/mucoco/models_bak_contd/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
model, tokenizer = define_model(ckpt_path, output_attentions=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

DEVICE:  cuda


In [5]:
# # 논문에서는 gpt-2의 tokenizer를 사용하였으므로, mask token이 기존에는 없었음
# tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

([50256], ['<|endoftext|>'], 50257)

In [6]:
# # tokenizer에 mask token 추가
# SPECIAL_TOKENS = {"mask_token": "<mask>"}
# tokenizer.add_special_tokens(SPECIAL_TOKENS)

1

In [7]:
# # 논문에서는 gpt-2의 tokenizer를 사용하였으므로, mask token이 기존에는 없었음
# tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

([50256, 50257], ['<|endoftext|>', '<mask>'], 50257)

In [8]:
# verify if the code is correct
test_sent = sample1['text'].tolist()[0:10]
batch = tokenizer(test_sent, padding=True, return_tensors="pt", truncation=True)

In [9]:
# cls token이 따로 없는데 잘 학습이 된게 맞을까? -> 상관없다고 하심 (교수님)
tokenizer.decode(batch['input_ids'][0])

"Ha ha, HILLARY LOST. All you hillary fools need to be rounded up and put in prison along with your leader. You are all anti--American scum. Perhaps we should dump you in Mexico along with all the illegal trash you let in....if you love 'em so much, go live with 'em. But you are not welcome here, you hate filled Marxist morons.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoft

In [10]:
classifier=model

In [11]:
torch.cuda.empty_cache()
# forward
classifier_output = classifier.forward(
    batch["input_ids"].cuda(),
    attention_mask=batch["attention_mask"].cuda(),
)
torch.cuda.empty_cache()

In [12]:
# get attentions
attentions = classifier_output["attentions"]
# attention_mask에서 1의 개수를 셈
lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
# 보고자 하는 attention layer 만 가져옴
attentions = attentions[
    10 # originally 10
]

In [18]:
cls_attns = attentions.max(1)[0][:, 0]

In [22]:
locate_ixes=[]
for i, attn in enumerate(cls_attns):
    # attention_mask가 1인 곳 까지의 attention을 보고, start of sentence와 end of sentence에 해당하는 token을 제거하고, softmax를 취한다.
    # current_attn = attn[: lengths[i]][1:-1].softmax(-1)
    current_attn = attn[: lengths[i]].softmax(-1) # <- current tokenizer does not add <s> and </s> to the sentence.
    # 이 값의 평균을 구한다.
    avg_value = current_attn.view(-1).mean().item()
    # 이 값 중에 평균보다 큰 값을 지니는 위치를 찾는다. (+1 because we skipped the first token)
    # top_masks = ((current_attn > avg_value).nonzero().view(-1)) + 1
    top_masks = ((current_attn > avg_value).nonzero().view(-1))
    torch.cuda.empty_cache()
    top_masks = top_masks.cpu().tolist()
    # attention 값이 평균보다 큰 토큰의 수가 6 또는 문장 전체 토큰 수의 1/3 보다 크면  
    if len(top_masks) > min((lengths[i] - 2) // 3, 6):
        # 그냥 attention 값 기준 top k 개 (k = 6 또는 토큰 수/3)를 뽑는다.
        top_masks = (
            # current_attn.topk(min((lengths[i] - 2) // 3, 6))[1] + 1
            current_attn.topk(min((lengths[i] - 2) // 3, 6))[1]
        )
        top_masks = top_masks.cpu().tolist()
    # 현재 문장의 input id를 가져온다.
    current_sent = batch["input_ids"][i][: lengths[i]]
    count = 0
    top_masks_final = []
    # top_masks에 뽑힌 index를 돌면서
    for index in top_masks:
        # mask해야 할 토큰이 and, of, or, so 에 해당하지 않으면
        if tokenizer.decode(current_sent[index]) not in [ ## maybe add more!
            " and",
            " of",
            " or",
            " so",
        ]:
            # token을 mask 한다.
            # current_sent[index] = mask_token
            top_masks_final.append(index)
            # count 수를 늘린다.
            count += 1
        else:
            # 만약에 and, of, or, so 에 해당하면 아무것도 하지 않는다.
            pass
    locate_ixes.append(top_masks_final)

In [23]:
locate_ixes

[[78, 75, 77, 79, 76, 80],
 [2, 0, 3, 1],
 [1, 2],
 [16, 0, 17, 10, 7, 8],
 [0, 2, 10, 13],
 [52, 19, 54, 17, 47, 53],
 [39, 40, 36, 34, 41, 30],
 [0, 13, 22, 25, 26, 27],
 [31, 30, 0, 29, 2, 28],
 [137, 135, 136, 133, 1, 0]]

### Find indices for bak_outputs.txt.init
- added on 23/07/18

In [5]:
samples=pd.read_json('outputs/toxicity/save-init-gen/bak_outputs.txt.init', lines=True)

In [7]:
# load trained model
ckpt_path='models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
model, tokenizer = define_model(ckpt_path, output_attentions=True, device=DEVICE)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

DEVICE:  cpu


In [12]:
test_sent = samples['generation'].tolist()
batch = tokenizer(test_sent, padding=True, return_tensors="pt", truncation=True)

In [13]:
# cls token이 따로 없는데 잘 학습이 된게 맞을까? -> 상관없다고 하심 (교수님)
tokenizer.decode(batch['input_ids'][0])

' Mr. Obama, and have been viewed by some as a bid to entice people back into the<|endoftext|>'

In [23]:
# torch.cuda.empty_cache()
# forward
model.eval()
with torch.no_grad():
    classifier_output = model.forward(**batch)
    torch.cuda.empty_cache()

In [24]:
# get attentions
attentions = classifier_output["attentions"]
# attention_mask에서 1의 개수를 셈
lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
# 보고자 하는 attention layer 만 가져옴
attentions = attentions[
    10 # originally 10
]

In [26]:
cls_attns = attentions.max(1)[0][:, 0]

In [34]:
locate_ixes=[]
for i, attn in tqdm(enumerate(cls_attns)):
    # attention_mask가 1인 곳 까지의 attention을 보고, start of sentence와 end of sentence에 해당하는 token을 제거하고, softmax를 취한다.
    # current_attn = attn[: lengths[i]][1:-1].softmax(-1)
    current_attn = attn[: lengths[i]].softmax(-1) # <- current tokenizer does not add <s> and </s> to the sentence.
    # 이 값의 평균을 구한다.
    avg_value = current_attn.view(-1).mean().item()
    # 이 값 중에 평균보다 큰 값을 지니는 위치를 찾는다. (+1 because we skipped the first token)
    # top_masks = ((current_attn >= avg_value).nonzero().view(-1)) + 1
    top_masks = ((current_attn >= avg_value).nonzero().view(-1)) ## fixed to reflect that sometimes the sequence length is 1.
    torch.cuda.empty_cache()
    top_masks = top_masks.cpu().tolist()
    if len(current_attn)==1:
        print(current_attn, top_masks, lengths[i])
    
    # attention 값이 평균보다 큰 토큰의 수가 6 또는 문장 전체 토큰 수의 1/3 보다 크면  
#     if len(top_masks) > min((lengths[i] - 2) // 3, 6):
    if len(top_masks) > min((lengths[i]) // 3, 6):
        # 그냥 attention 값 기준 top k 개 (k = 6 또는 토큰 수/3)를 뽑는다.
        top_masks = (
            # current_attn.topk(min((lengths[i] - 2) // 3, 6))[1] + 1
#             current_attn.topk(min((lengths[i] - 2) // 3, 6))[1]
            current_attn.topk(max(min((lengths[i]) // 3, 6), 1))[1]
        )
        top_masks = top_masks.cpu().tolist()
        if len(current_attn) == 1:
            print("top_masks after selecting top k", top_masks)
    # 현재 문장의 input id를 가져온다.
    current_sent = batch["input_ids"][i][: lengths[i]]
    count = 0
    top_masks_final = []
    # top_masks에 뽑힌 index를 돌면서
    for index in top_masks:
        # mask해야 할 토큰이 and, of, or, so 에 해당하지 않으면
        if tokenizer.decode(current_sent[index]) not in [ ## maybe add more!
            " and",
            " of",
            " or",
            " so",
        ]:
            # token을 mask 한다.
            # current_sent[index] = mask_token
            top_masks_final.append(index)
            # count 수를 늘린다.
            count += 1
        else:
            # 만약에 and, of, or, so 에 해당하면 아무것도 하지 않는다.
            pass
    locate_ixes.append(top_masks_final)

2375it [00:00, 9613.68it/s]

tensor([1.]) [0] 1
top_masks after selecting top k [0]


In [35]:
locate_ixes

[[2, 5, 1, 7, 0, 19],
 [0, 12, 13, 16, 17, 18],
 [0, 1, 3, 9, 16, 19],
 [0, 1, 2, 8, 17],
 [0, 1, 2, 14, 18, 19],
 [0, 3, 7, 9],
 [0, 10, 11, 15, 16],
 [7, 0, 18, 16, 1, 11],
 [17, 19, 13, 0, 12, 18],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 3, 4, 5, 7, 10],
 [0, 1, 8, 9, 10],
 [8, 1, 0, 6, 11, 18],
 [0, 1, 10, 2, 12, 13],
 [0, 1, 7, 10, 17, 19],
 [0, 3, 8, 15, 16, 17],
 [0, 11, 4, 3, 16, 15],
 [0, 8, 12, 14, 15, 19],
 [4, 1, 6, 3, 19, 5],
 [0, 1, 7, 11, 12],
 [0, 3, 4, 18, 17],
 [3, 1, 18, 16, 15],
 [0, 2, 3, 10, 12],
 [2, 0, 4, 1, 3, 17],
 [3, 4, 0, 5, 13, 18],
 [14, 5, 3, 0, 12, 1],
 [0, 5, 14, 11, 1, 13],
 [0, 2, 4, 5, 17, 18],
 [3, 4, 0, 7, 6],
 [0, 1, 2, 4, 9, 17],
 [19, 11, 13, 14, 10, 18],
 [0, 1, 4, 6, 17, 19],
 [0, 1, 4, 11],
 [19, 2, 11, 0, 10, 9],
 [0, 11, 3, 4, 15, 5],
 [0, 1, 2, 3, 4, 19],
 [0, 3, 6, 17, 18],
 [0, 3, 10, 17],
 [6, 18, 0, 7, 17],
 [0, 1, 6, 9, 15],
 [10, 0, 3, 2, 1, 5],
 [5, 1, 13, 2, 4, 0],
 [0, 2, 13, 14, 15, 16],
 [2, 3, 0, 12, 13, 10],
 [0, 1, 5, 6, 11, 19],

In [37]:
samples['indices'] = locate_ixes

In [38]:
samples

,prompt,generation,indices
0,These stunts are the latest in a long line of ...,"Mr. Obama, and have been viewed by some as a ...","[2, 5, 1, 7, 0, 19]"
1,These stunts are the latest in a long line of ...,"the real-life Eubank Khorshid, or Mr Disappea...","[0, 12, 13, 16, 17, 18]"
2,These stunts are the latest in a long line of ...,German artists who demand visibility for grap...,"[0, 1, 3, 9, 16, 19]"
3,These stunts are the latest in a long line of ...,"these dolphin hunters who hunt these ""native""...","[0, 1, 2, 8, 17]"
4,These stunts are the latest in a long line of ...,"her female adversaries. Last year, Women in t...","[0, 1, 2, 14, 18, 19]"
...,...,...,...
2370,"With 2.3 billion adherents, Christians are the...",in the future. This rapid growth was the deci...,"[0, 1, 2, 3, 16, 17]"
2371,"With 2.3 billion adherents, Christians are the...",as hundreds of millions more Christians go on...,"[5, 9, 12, 13, 14, 18]"
2372,"With 2.3 billion adherents, Christians are the...",. Only a bit more than 1 in 4 Buddhists and Hi...,"[17, 9, 16, 12, 1, 6]"
2373,"With 2.3 billion adherents, Christians are the...",through at least 2050—17 million more per yea...,"[16, 2, 0, 7, 4, 3]"


In [40]:
samples.to_json('outputs/toxicity/save-init-gen/bak_outputs.txt.init.widx', orient="records", lines=True)

### 함수화

In [ ]:
# model 의 forward 함수에서 정의를 output_attentions=True를 넘길 수 있게 되어 있다.
def locate(model, tokenizer, batch, num_layer=10):
    # torch.cuda.empty_cache()
    # forward
    model.eval()
    with torch.no_grad():
        classifier_output = model.forward(**batch, output_attentions=True)
        torch.cuda.empty_cache()
        
    # get attentions
    attentions = classifier_output["attentions"]
    # attention_mask에서 1의 개수를 셈
    lengths = [i.tolist().count(1) for i in batch["attention_mask"]]
    # 보고자 하는 attention layer 만 가져옴
    attentions = attentions[
        num_layer # originally 10
    ]
    cls_attns = attentions.max(1)[0][:, 0]
    
    locate_ixes=[]
    for i, attn in tqdm(enumerate(cls_attns)):
        # attention_mask가 1인 곳 까지의 attention을 보고, start of sentence와 end of sentence에 해당하는 token을 제거하고, softmax를 취한다.
        # current_attn = attn[: lengths[i]][1:-1].softmax(-1)
        current_attn = attn[: lengths[i]].softmax(-1) # <- current tokenizer does not add <s> and </s> to the sentence.
        # 이 값의 평균을 구한다.
        avg_value = current_attn.view(-1).mean().item()
        # 이 값 중에 평균보다 큰 값을 지니는 위치를 찾는다. (+1 because we skipped the first token)
        # top_masks = ((current_attn >= avg_value).nonzero().view(-1)) + 1
        top_masks = ((current_attn >= avg_value).nonzero().view(-1)) ## fixed to reflect that sometimes the sequence length is 1.
        torch.cuda.empty_cache()
        top_masks = top_masks.cpu().tolist()
        if len(current_attn)==1:
            print(current_attn, top_masks, lengths[i])
        
        # attention 값이 평균보다 큰 토큰의 수가 6 또는 문장 전체 토큰 수의 1/3 보다 크면  
    #     if len(top_masks) > min((lengths[i] - 2) // 3, 6):
        if len(top_masks) > min((lengths[i]) // 3, 6):
            # 그냥 attention 값 기준 top k 개 (k = 6 또는 토큰 수/3)를 뽑는다.
            top_masks = (
                # current_attn.topk(min((lengths[i] - 2) // 3, 6))[1] + 1
    #             current_attn.topk(min((lengths[i] - 2) // 3, 6))[1]
                current_attn.topk(max(min((lengths[i]) // 3, 6), 1))[1]
            )
            top_masks = top_masks.cpu().tolist()
            if len(current_attn) == 1:
                print("top_masks after selecting top k", top_masks)
        # 현재 문장의 input id를 가져온다.
        current_sent = batch["input_ids"][i][: lengths[i]]
        count = 0
        top_masks_final = []
        # top_masks에 뽑힌 index를 돌면서
        for index in top_masks:
            # mask해야 할 토큰이 and, of, or, so 에 해당하지 않으면
            if tokenizer.decode(current_sent[index]) not in [ ## maybe add more!
                " and",
                " of",
                " or",
                " so",
            ]:
                # token을 mask 한다.
                # current_sent[index] = mask_token
                top_masks_final.append(index)
                # count 수를 늘린다.
                count += 1
            else:
                # 만약에 and, of, or, so 에 해당하면 아무것도 하지 않는다.
                pass
        locate_ixes.append(top_masks_final)
    return locate_ixes